<a href="https://colab.research.google.com/github/RyanSilva2004/AI-Driven-JobFinder-.Net/blob/main/Sinhala-English-LLM-Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import re
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Sinhala LLM Task 1/dataset.csv')
df.head()

,Sinhala,English,Singlish
0,c esc ma tete මෙය මගේ ප්‍රධාන අයිතියයි ඔබ ප්‍ර...,c esc ma tete this is my headright see you re ...,[Unkown] [Unkown] [Unkown] [Unkown] meya mage ...
1,නෑ නෑ ඒක මගේ වරදක් අපිට හරි හමන් කැමරොන් කෙනෙක...,no no it s my fault we didn t have a proper in...,ne ne eka mage waradak apita hari haman [Unkow...
2,හැක් කිරීම සහ කට වහගෙන කෙළ ගසන කොටස නොවේ කරුණා...,not the hacking and gagging and spitting part ...,hek kireema saha kata wahagena kela gasana kot...
3,"හොඳයි, මම හිතුවා අපි උච්චාරණයෙන් පටන් ගනිමු එය...",well i thought we d start with pronunciation i...,hondayi mama hithuwa api uchcharanayen patan g...
4,ඔයා මගෙන් අහනවා ඒක හරිම ලස්සනයි ඔයාගේ නම මොකක්...,you re asking me out that s so cute what s you...,oya magen ahanawa eka harima lassanayi oyage n...


In [ ]:
import os
folder_path = '/content/drive/MyDrive/Sinhala LLM Task 1/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

df_sinhala = df[['Sinhala']]
df_english = df[['English']]
df_singlish = df[['Singlish']]
df_sinhala.to_csv('/content/drive/MyDrive/Sinhala LLM Task 1/Sinhala.csv', index=False)
df_english.to_csv('/content/drive/MyDrive/Sinhala LLM Task 1/English.csv', index=False)
df_singlish.to_csv('/content/drive/MyDrive/Sinhala LLM Task 1/Singlish.csv', index=False)


In [ ]:
df_english['English'] = df_english['English'].str.lower()
df_english.to_csv('/content/drive/MyDrive/Sinhala LLM Task 1/EnglishClean.csv', index=False)

def remove_english_letters(text):
    return re.sub(r'[A-Za-z]', '', text)
df_sinhala['Sinhala'] = df_sinhala['Sinhala'].apply(remove_english_letters)
df_sinhala.to_csv('/content/drive/MyDrive/Sinhala LLM Task 1/SinhalaClean.csv', index=False)

<ipython-input-106-d014167c2c3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['English'] = df_english['English'].str.lower()
<ipython-input-106-d014167c2c3c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinhala['Sinhala'] = df_sinhala['Sinhala'].apply(remove_english_letters)


In [ ]:

english_file = '/content/drive/MyDrive/Sinhala LLM Task 1/EnglishClean.csv'
sinhala_file = '/content/drive/MyDrive/Sinhala LLM Task 1/SinhalaClean.csv'

START_TOKEN = '<START>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<END>'

sinhala_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      ':', '<', '=', '>', '?', '@',
                      'අ', 'ආ', 'ඇ', 'ඈ', 'ඉ', 'ඊ', 'උ', 'ඌ', 'එ', 'ඒ', 'ඔ', 'ඕ', 'ඖ', 'ක', 'ඛ', 'ග', 'ඝ', 'ඞ', 'ඟ',
                      'ච', 'ඡ', 'ජ', 'ඣ', 'ඤ', 'ඥ', 'ඦ', 'ට', 'ඨ', 'ඩ', 'ඪ', 'ණ', 'ඬ', 'ත', 'ථ', 'ද', 'ධ', 'න', 'ඳ',
                      'ප', 'ඵ', 'බ', 'භ', 'ම', 'ඹ', 'ය', 'ර', 'ල', 'ව', 'ශ', 'ෂ', 'ස', 'හ', 'ළ', 'ෆ', 'ක්‍ෂ',
                      'ා', 'ැ', 'ෑ', 'ි', 'ී', 'ු', 'ූ', 'ෙ', 'ේ', 'ෛ', 'ො', 'ෝ', 'ෞ', '්', 'ා', '‍ර', '‍ය', 'ෲ',
                      '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                      'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                      'y', 'z',
                      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
                      'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
                      'Y', 'Z', '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]


In [ ]:
text = 'නෑ නෑ ඒක මගේ වරදක්'
list(text)

['න',
 'ෑ',
 ' ',
 'න',
 'ෑ',
 ' ',
 'ඒ',
 'ක',
 ' ',
 'ම',
 'ග',
 'ේ',
 ' ',
 'ව',
 'ර',
 'ද',
 'ක',
 '්']

In [ ]:
def clean_sinhala_text(text):
    return text.replace('\u200d', '')

In [ ]:
index_to_sinhala = {k: clean_sinhala_text(v) for k, v in enumerate(sinhala_vocabulary) if v}
sinhala_to_index = {v: k for k, v in index_to_sinhala.items()}
index_to_english = {k: v for k, v in enumerate(english_vocabulary)}
english_to_index = {v: k for k, v in index_to_english.items()}

In [ ]:
print (index_to_sinhala)

{0: '<START>', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9', 27: ':', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: 'අ', 34: 'ආ', 35: 'ඇ', 36: 'ඈ', 37: 'ඉ', 38: 'ඊ', 39: 'උ', 40: 'ඌ', 41: 'එ', 42: 'ඒ', 43: 'ඔ', 44: 'ඕ', 45: 'ඖ', 46: 'ක', 47: 'ඛ', 48: 'ග', 49: 'ඝ', 50: 'ඞ', 51: 'ඟ', 52: 'ච', 53: 'ඡ', 54: 'ජ', 55: 'ඣ', 56: 'ඤ', 57: 'ඥ', 58: 'ඦ', 59: 'ට', 60: 'ඨ', 61: 'ඩ', 62: 'ඪ', 63: 'ණ', 64: 'ඬ', 65: 'ත', 66: 'ථ', 67: 'ද', 68: 'ධ', 69: 'න', 70: 'ඳ', 71: 'ප', 72: 'ඵ', 73: 'බ', 74: 'භ', 75: 'ම', 76: 'ඹ', 77: 'ය', 78: 'ර', 79: 'ල', 80: 'ව', 81: 'ශ', 82: 'ෂ', 83: 'ස', 84: 'හ', 85: 'ළ', 86: 'ෆ', 87: 'ක්ෂ', 88: 'ා', 89: 'ැ', 90: 'ෑ', 91: 'ි', 92: 'ී', 93: 'ු', 94: 'ූ', 95: 'ෙ', 96: 'ේ', 97: 'ෛ', 98: 'ො', 99: 'ෝ', 100: 'ෞ', 101: '්', 102: 'ා', 103: 'ර', 104: 'ය', 105: 'ෲ', 106: '{', 107: '|', 108: '}', 109: '~', 1

In [ ]:
print (index_to_english)

{0: '<START>', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '$', 6: '%', 7: '&', 8: "'", 9: '(', 10: ')', 11: '*', 12: '+', 13: ',', 14: '-', 15: '.', 16: '/', 17: '0', 18: '1', 19: '2', 20: '3', 21: '4', 22: '5', 23: '6', 24: '7', 25: '8', 26: '9', 27: ':', 28: ';', 29: '<', 30: '=', 31: '>', 32: '?', 33: '@', 34: '[', 35: '\\', 36: ']', 37: '^', 38: '_', 39: '`', 40: 'a', 41: 'b', 42: 'c', 43: 'd', 44: 'e', 45: 'f', 46: 'g', 47: 'h', 48: 'i', 49: 'j', 50: 'k', 51: 'l', 52: 'm', 53: 'n', 54: 'o', 55: 'p', 56: 'q', 57: 'r', 58: 's', 59: 't', 60: 'u', 61: 'v', 62: 'w', 63: 'x', 64: 'y', 65: 'z', 66: 'A', 67: 'B', 68: 'C', 69: 'D', 70: 'E', 71: 'F', 72: 'G', 73: 'H', 74: 'I', 75: 'J', 76: 'K', 77: 'L', 78: 'M', 79: 'N', 80: 'O', 81: 'P', 82: 'Q', 83: 'R', 84: 'S', 85: 'T', 86: 'U', 87: 'V', 88: 'W', 89: 'X', 90: 'Y', 91: 'Z', 92: '{', 93: '|', 94: '}', 95: '~', 96: '<PAD>', 97: '<END>'}


In [ ]:
# Tokenizing Sinhala and English sentences
def tokenize_sentence(sentence, token_to_index):
    return [token_to_index[char] for char in sentence if char in token_to_index]

# Tokenizing Sinhala and English datasets
df['tokenized_sinhala'] = df['Sinhala'].apply(lambda x: tokenize_sentence(x, sinhala_to_index))
df['tokenized_english'] = df['English'].apply(lambda x: tokenize_sentence(x, english_to_index))

# Checking the tokenized data
df[['tokenized_sinhala', 'tokenized_english']].head()


,tokenized_sinhala,tokenized_english
0,"[1, 1, 1, 1, 75, 95, 104, 1, 75, 48, 96, 1, 71...","[42, 1, 44, 58, 42, 1, 52, 40, 1, 59, 44, 59, ..."
1,"[69, 90, 1, 69, 90, 1, 42, 46, 1, 75, 48, 96, ...","[53, 54, 1, 53, 54, 1, 48, 59, 1, 58, 1, 52, 6..."
2,"[84, 89, 46, 101, 1, 46, 91, 103, 92, 75, 1, 8...","[53, 54, 59, 1, 59, 47, 44, 1, 47, 40, 42, 50,..."
3,"[84, 98, 70, 104, 91, 13, 1, 75, 75, 1, 84, 91...","[62, 44, 51, 51, 1, 48, 1, 59, 47, 54, 60, 46,..."
4,"[43, 104, 102, 1, 75, 48, 95, 69, 101, 1, 33, ...","[64, 54, 60, 1, 57, 44, 1, 40, 58, 50, 48, 53,..."


In [ ]:
# Find the maximum length of tokenized sentences
max_len_sinhala = max(df['tokenized_sinhala'].apply(len))
max_len_english = max(df['tokenized_english'].apply(len))

# Use the larger of the two as the max sequence length
max_seq_len = max(max_len_sinhala, max_len_english)
print(f"Max sequence length: {max_seq_len}")


Max sequence length: 429


In [ ]:
# Sinhala Tokenization based on words (splitting by space)
df['tokenized_sinhala'] = df['Sinhala'].apply(lambda x: x.split())

# English Tokenization based on words (splitting by space)
df['tokenized_english'] = df['English'].apply(lambda x: x.split())

# Check the tokenized output
df[['tokenized_sinhala', 'tokenized_english']].head()


,tokenized_sinhala,tokenized_english
0,"[c, esc, ma, tete, මෙය, මගේ, ප්‍රධාන, අයිතියයි...","[c, esc, ma, tete, this, is, my, headright, se..."
1,"[නෑ, නෑ, ඒක, මගේ, වරදක්, අපිට, හරි, හමන්, කැමර...","[no, no, it, s, my, fault, we, didn, t, have, ..."
2,"[හැක්, කිරීම, සහ, කට, වහගෙන, කෙළ, ගසන, කොටස, න...","[not, the, hacking, and, gagging, and, spittin..."
3,"[හොඳයි,, මම, හිතුවා, අපි, උච්චාරණයෙන්, පටන්, ග...","[well, i, thought, we, d, start, with, pronunc..."
4,"[ඔයා, මගෙන්, අහනවා, ඒක, හරිම, ලස්සනයි, ඔයාගේ, ...","[you, re, asking, me, out, that, s, so, cute, ..."


In [ ]:
# Function to convert tokenized sentence to index-based sequence
def words_to_indices(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index[PADDING_TOKEN] for word in sentence]

# Convert Sinhala and English tokenized sentences to indexed sequences
df['indexed_sinhala'] = df['tokenized_sinhala'].apply(lambda x: words_to_indices(x, sinhala_to_index))
df['indexed_english'] = df['tokenized_english'].apply(lambda x: words_to_indices(x, english_to_index))

# Check the indexed sequences
df[['indexed_sinhala', 'indexed_english']].head()


,indexed_sinhala,indexed_english
0,"[618, 938, 1480, 2200, 21488, 19993, 16579, 33...","[2146, 4960, 8764, 14614, 14710, 7658, 9674, 6..."
1,"[14589, 14589, 6225, 19993, 24009, 3158, 29033...","[9924, 9924, 7674, 12424, 9674, 5302, 15900, 4..."
2,"[29221, 8418, 26915, 6637, 24177, 8927, 9631, ...","[9974, 14642, 6482, 669, 5932, 669, 13700, 105..."
3,"[29934, 20172, 29599, 3157, 5389, 15686, 9491,...","[15961, 7207, 14724, 15900, 3671, 13854, 16155..."
4,"[6408, 19991, 3870, 6225, 29055, 23166, 6414, ...","[16357, 11740, 961, 9086, 10337, 14631, 12424,..."


In [ ]:
# Padding function to pad a sequence to the desired length
def pad_sequence(sequence, max_len, padding_token):
    return sequence + [sinhala_to_index[padding_token]] * (max_len - len(sequence))

# Find the maximum length of tokenized sentences
max_len_sinhala = max(df['indexed_sinhala'].apply(len))
max_len_english = max(df['indexed_english'].apply(len))

# Use the larger of the two as the max sequence length
max_seq_len = max(max_len_sinhala, max_len_english)
print(f"Max sequence length: {max_seq_len}")

# Padding the indexed Sinhala and English sentences
df['padded_sinhala'] = df['indexed_sinhala'].apply(lambda x: pad_sequence(x, max_seq_len, PADDING_TOKEN))
df['padded_english'] = df['indexed_english'].apply(lambda x: pad_sequence(x, max_seq_len, PADDING_TOKEN))

# Check the padded sequences
df[['padded_sinhala', 'padded_english']].head()


Max sequence length: 62


,padded_sinhala,padded_english
0,"[618, 938, 1480, 2200, 21488, 19993, 16579, 33...","[2146, 4960, 8764, 14614, 14710, 7658, 9674, 6..."
1,"[14589, 14589, 6225, 19993, 24009, 3158, 29033...","[9924, 9924, 7674, 12424, 9674, 5302, 15900, 4..."
2,"[29221, 8418, 26915, 6637, 24177, 8927, 9631, ...","[9974, 14642, 6482, 669, 5932, 669, 13700, 105..."
3,"[29934, 20172, 29599, 3157, 5389, 15686, 9491,...","[15961, 7207, 14724, 15900, 3671, 13854, 16155..."
4,"[6408, 19991, 3870, 6225, 29055, 23166, 6414, ...","[16357, 11740, 961, 9086, 10337, 14631, 12424,..."
